**<h1>Setup and data processing<h1>**

---



<h3>Package installation<h3>


In [ ]:
%pip install pandas
%pip install python-binance
%pip install ta
%pip install matplotlib
%pip install numpy

<h3>Imports<h3>

In [ ]:
import pandas as pd
from binance.client import Client
import ta
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

<h3>Specs<h3>

In [ ]:
name_base = "ETH"
name_quote = "USDT"
timeframe = "1d"

starting_date = "01 january 2017"
ending_date = None

initial_wallet = 1000
trade_fees = 0.001

<h3>Binance data<h3>

In [ ]:
# download the coin info
symbol = name_base+name_quote
info = Client().get_historical_klines(symbol, timeframe, starting_date)
# storing it into a pandas data frame
data_dl = pd.DataFrame(info, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
data_dl

In [ ]:
# keeping only the necessary columns
data_dl.drop(columns=data_dl.columns.difference(['timestamp','close']), inplace=True)
# formating the index
data_dl.set_index(data_dl['timestamp'], inplace=True)
data_dl.index = pd.to_datetime(data_dl.index, unit='ms')
del data_dl['timestamp']
# formating the numbers
data_dl["close"] = pd.to_numeric(data_dl["close"])
data_dl

**<h1>Optimisation<h1>**


---

<h3>Buy-Sell condition functions<h3>

In [ ]:
data_dl['RSI'] = ta.momentum.rsi(data_dl['close'])


def buy_condition(row, previous_row):
    return row['MAs'] > row['MAl'] and row['RSI'] < 70
    
def sell_condition(row, previous_row):
    return row['MAs'] < row['MAl'] and row['RSI'] > 30

<h3>Loop<h3>

In [ ]:
array_mas = np.linspace(1, 30, int((30-1)/1+1), dtype=int)
array_mal = np.linspace(10, 40, int((40-10)/1+1), dtype=int)

results = []
for mas in array_mas:
    for mal in array_mal:
        if mal > mas:
            
            ### indicators
            data = data_dl.copy()
            data['MAs'] = ta.trend.ema_indicator(data['close'], mas)
            data['MAl'] = ta.trend.ema_indicator(data['close'], mal)
            data.dropna(inplace=True)

            ### initialisations
            quote = initial_wallet
            base = 0
            orders = []
            data['wallet'] = ''
            last_ath = 0
            previous_row = data.iloc[0]

            ### backtest loop
            for index, row in data.iterrows():

                value = row['close']

                # buy
                if buy_condition(row, previous_row) and quote > 0:
                    base = quote / value
                    fee = base * trade_fees
                    base -= fee
                    fee *= value
                    quote = 0
                    wallet = base * value

                    if wallet > last_ath:
                        last_ath = wallet

                    orders.append({'side': 'buy',
                                   'wallet': wallet,
                                   'fee': fee,
                                   'drawdown': (wallet - last_ath) / last_ath,
                                   })

                # sell
                elif sell_condition(row, previous_row) and base > 0:
                    quote = base * value
                    fee = quote * trade_fees
                    quote -= fee
                    base = 0
                    wallet = quote

                    if wallet > last_ath:
                        last_ath = wallet

                    orders.append({'side': 'sell',
                                   'wallet': wallet,
                                   'fee': fee,
                                   'drawdown': (wallet - last_ath) / last_ath,
                                   })
                    
                # tracking the wallet value
                if quote == 0:
                    data.at[index, 'wallet'] = base * value
                else:
                    data.at[index, 'wallet'] = quote

                previous_row = row


            ### analysing the trades
            orders = pd.DataFrame(orders, columns=['side', 'wallet', 'fee', 'drawdown'])
            orders['PnL%'] = orders['wallet'].pct_change()*100
            orders.at[0, 'PnL%'] = (orders.iloc[0]['wallet']-initial_wallet)/initial_wallet*100
            orders.loc[orders['side']=='buy','PnL%'] = None
            orders['Win'] = ''
            orders.loc[orders['PnL%']>0,'Win'] = 'Yes'
            orders.loc[orders['PnL%']<=0,'Win'] = 'No'
            if 'No' in orders['Win'].values and 'Yes' in orders['Win'].values:
                n_pos = orders['Win'].value_counts()['Yes']
                n_neg = orders['Win'].value_counts()['No']
                n_trades = n_neg + n_pos
                winrate = n_pos / n_trades * 100
            elif 'No' not in orders['Win'].values and 'Yes' not in orders['Win'].values:
                n_trades = 0
                winrate = 0
            elif 'Yes' not in orders['Win'].values:
                n_trades = orders['Win'].value_counts()['No']
            else:
                n_trades = orders['Win'].value_counts()['Yes']
                winrate = 100

            res = {
                  'mas': mas,
                  'mal': mal,
                  'wallet': data.iloc[-1]['wallet'],
                  'profits': 100 * (data.iloc[-1]['wallet']-initial_wallet)/initial_wallet,
                  'ntrades': round(n_trades,1),
                  'winrate': winrate,
                  'avg_wins': round(orders.loc[orders['Win'] == 'Yes']['PnL%'].mean(),2),
                  'avg_loses': round(orders.loc[orders['Win'] == 'No']['PnL%'].mean(),2),
                  'drawdown_max': round(orders['drawdown'].min()*100,2),
                  'fee':round(orders['fee'].sum(),2),
            }
            results.append(res)

            print(f"MA-st = {res['mas']}; MA-lt = {res['mal']}; Profits = {round(res['profits'],2)}%")

 

**<h1>Results<h1>**


---



In [ ]:
# results into a dataframe
results = pd.DataFrame(results, columns=['mas', 'mal', 'wallet', 'profits', 'ntrades', 'winrate', 'avg_wins', 'avg_loses', 'drawdown_max', 'fee'])

# color map for plots
cmap_reversed = cm.get_cmap('PuOr_r')

<h3>Profits<h3>

In [ ]:
# number of trades
plt.figure(figsize=(6, 4))
sc = plt.scatter(results["mas"], results["mal"], c=results["ntrades"], cmap=cmap_reversed, s=25)
plt.xlabel("Short term EMA", fontsize=22)
plt.ylabel("Long term EMA", fontsize=22)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
cb = plt.colorbar(sc)
cb.set_label(label = 'N trades', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.clim(0,200)
plt.tight_layout()

# profits
plt.figure(figsize=(6, 4))
sc = plt.scatter(results["mas"], results["mal"], c=results['profits'], cmap=cmap_reversed, s=25)
plt.xlabel("Short term EMA", fontsize=22)
plt.ylabel("Long term EMA", fontsize=22)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
cb = plt.colorbar(sc)
cb.set_label(label = 'Profits (%)', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.clim(0,8000)
plt.tight_layout()

# wallet
plt.figure(figsize=(6, 4))
sc = plt.scatter(results["mas"], results["mal"], c=results["wallet"], cmap=cmap_reversed, s=25)
plt.xlabel("Short term EMA", fontsize=22)
plt.ylabel("Long term EMA", fontsize=22)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
cb = plt.colorbar(sc)
cb.set_label(label = f'Wallet {name_quote}', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.clim(0,80000)
plt.tight_layout()

<h3>Winrate<h3>

In [ ]:
# winrate
plt.figure(figsize=(6, 4))
sc = plt.scatter(results["mas"], results["mal"], c=results["winrate"], cmap=cmap_reversed, s=25)
plt.xlabel("Short term EMA", fontsize=22)
plt.ylabel("Long term EMA", fontsize=22)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
cb = plt.colorbar(sc)
cb.set_label(label = 'Winrate (%)', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.clim(20,60)
plt.tight_layout()

# average winning trades profits 
plt.figure(figsize=(6, 4))
sc = plt.scatter(results["mas"], results["mal"], c=results["avg_wins"], cmap=cmap_reversed, s=25)
plt.xlabel("Short term EMA", fontsize=22)
plt.ylabel("Long term EMA", fontsize=22)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
cb = plt.colorbar(sc)
cb.set_label(label = 'Average Win Profits(%)', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.clim(0,200)
plt.tight_layout()

# average loosing trades loses
plt.figure(figsize=(6, 4))
sc = plt.scatter(results["mas"], results["mal"], c=results["avg_loses"], cmap=cmap_reversed, s=25)
plt.xlabel("Short term EMA", fontsize=22)
plt.ylabel("Long term EMA", fontsize=22)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
cb = plt.colorbar(sc)
cb.set_label(label = 'Average Lose Loses (%)', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.clim(-18,-2)
plt.tight_layout()

<h3>Worst Drawdown<h3>

In [ ]:
plt.figure(figsize=(6, 4))
sc = plt.scatter(results["mas"], results["mal"], c=results["drawdown_max"], cmap=cmap_reversed, s=25)
plt.xlabel("Short term EMA", fontsize=22)
plt.ylabel("Long term EMA", fontsize=22)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
cb = plt.colorbar(sc)
cb.set_label(label = 'Worst drawdown (%)', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.clim(-80,-20)
plt.tight_layout()

<h3>Fees<h3>

In [ ]:
plt.figure(figsize=(6, 4))
sc = plt.scatter(results["mas"], results["mal"], c=results["fee"], cmap=cmap_reversed, s=25)
plt.xlabel("Short term EMA", fontsize=22)
plt.ylabel("Long term EMA", fontsize=22)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
cb = plt.colorbar(sc)
cb.set_label(label = f'Fees {name_quote}', fontsize=22)
cb.ax.tick_params(labelsize=13)
plt.clim(200,2500)
plt.tight_layout()